In [ ]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS

def pairs_trading(data1, data2, capital, window=100, zscore_entry=2, zscore_exit=-2):
    # Calculate the rolling hedge ratio using OLS
    ols = RollingOLS(y=data2, x=data1, window=window, intercept=False)
    hedge_ratio = ols.fit().params[0]

    # Calculate the spread and the z-score
    spread = data2 - hedge_ratio * data1
    spread_mean = spread.rolling(window=window).mean()
    spread_std = spread.rolling(window=window).std()
    zscore = (spread - spread_mean) / spread_std

    # Generate trading signals
    entry_signal = np.where(zscore > zscore_entry, 1, 0)
    entry_signal = np.where(zscore < -zscore_entry, -1, entry_signal)
    exit_signal = np.where(zscore < zscore_exit, -1, 0)
    exit_signal = np.where(zscore > -zscore_exit, 1, exit_signal)
    trade_signal = entry_signal + exit_signal
    trade_signal = trade_signal.fillna(method='ffill')

    # Calculate the value of each instrument
    price1 = data1
    price2 = data2
    value1 = price1 * hedge_ratio
    value2 = price2
    value = value1 + value2

    # Update the quantities for each instrument
    positions = pd.DataFrame({'signal': trade_signal})
    positions['pos1'] = trade_signal * (capital / price1) * (value1 / value)
    positions['pos2'] = -1 * trade_signal * (capital / price2) * (value2 / value)

    return positions

In the code, the short position is taken in instrument 2 and the long position is taken in instrument 1 because the hedge ratio is calculated using instrument 1 as the explanatory variable and instrument 2 as the response variable. The hedge ratio represents the ratio of the investment in instrument 1 to the investment in instrument 2 to achieve a hedge or neutral position.

When the z-score is greater than the entry threshold, a long position is taken in instrument 1 and a short position is taken in instrument 2, so that the spread between the two instruments remains close to zero. On the other hand, when the z-score is less than the exit threshold, the long and short positions are closed, effectively returning the portfolio to a neutral position.

In this way, the code implements a pairs trading strategy where the goal is to profit from the spread between two instruments by taking long and short positions in them.


What would you suggest, 1 or 2?:

In [ ]:
1#
positions['pos1'] = trade_signal * (capital / price1) * (value1 / value)
positions['pos2'] = -1 * trade_signal * (capital / price2) * (value2 / value)

In [ ]:
2#
positions['pos1'] = trade_signal * (capital / price1) * (1 / 2)
positions['pos2'] = -1 * trade_signal * (capital / price2) * (1 / 2)

Obviously, option 1.

Option 1 calculates the positions based on the value of each instrument, taking into account the hedge ratio, which represents the ratio of investment in instrument 1 to instrument 2 to achieve a hedge or neutral position. By using the value of each instrument to calculate the positions, this approach ensures that the portfolio remains hedge or neutral, as desired in a pairs trading strategy.

The option 2 calculates the positions based on a fixed allocation, which may not result in a hedge or neutral position and may not align with the goals of a pairs trading strategy.